# Handout 5  

Bruce Schultz

### Exercise 1

##### Part a  

**Write a function single_fasta_sequence(filename) that reads a sequence given in
FASTA format from a file containing a single sequence.**

In [6]:
def single_fasta_sequence(file):
    """
    :param fasta_file: FASTA formatted file contents
    :return: Tuple containing the header in position 0 and sequence in position 1
    """
    content = file.read().splitlines()
    header = content[0][1:]  # Remove the '>' character
    sequence = ''.join(content[1:])  # Join sequence lines together
    return (header, sequence)

In [9]:
with open('ecoli-genome.fna', 'r') as f:
    hd, seq = single_fasta_sequence(f)
print(seq[:10])

AGCTTTTCAT


##### Part b  

**Write a function fasta_list(filename) that reads all sequences from a FASTA file and returns a list of tuples, each tuple containing the header as the first and the sequence as the second element.**

In [10]:
def fasta_list(file):
    """
    Same as single_fasta_sequence, only works on a file containing multiple FASTA sequences
    :param fasta_file: FASTA formatted file content with 1+ sequences
    :return: List of tuples, each containing the header in position 0 and sequence in position 1
    """
    content = file.read().split('>')[1:]
    genes = []
    for gene in content:
        gene = gene.splitlines()  # Split the lines
        header = gene[0]
        sequence = ''.join(gene[1:]).replace('\n', '')
        genes.append((header, sequence))
    return genes

In [12]:
with open('ecoli-genes.ffn', 'r') as f:
    genes = fasta_list(f)
print(genes[0][:50])

('gi|556503834|ref|NC_000913.3|:190-255 Escherichia coli str. K-12 substr. MG1655, complete genome', 'ATGAAACGCATTAGCACCACCATTACCACCACCATCACCATTACCACAGGTAACGGTGCGGGCTGA')


##### Part c  

**Write a function fasta_sequences(filename) that can be used in the manner described above using a self written generator using the yield keyword.**

In [19]:
def fasta_sequences(fasta_file):
    with open(fasta_file, 'r') as file:
        for gene in fasta_list(file):
            yield gene

In [22]:
name,seq = max(fasta_sequences("ecoli-genes.ffn"),key= lambda x: len(x[1]))
max_length = len(seq)
print("The longest gene is",name,"and contains",max_length,"nucleobases.")

The longest gene is gi|556503834|ref|NC_000913.3|:2044938-2052014 Escherichia coli str. K-12 substr. MG1655, complete genome and contains 7077 nucleobases.


##### Part d  

**Test the functions written in (b) and (c) on ecoli-proteome.faa . Read the Fasta file
and print the header and the length of the shortest and longest amino acid sequence.**

In [28]:
# Lists
with open('ecoli-proteome.faa', 'r') as f:
    genes = fasta_list(f)
name,seq = max(genes,key= lambda x: len(x[1]))
max_length = len(seq)
print("Lists:\nThe longest gene is",name,"and contains",max_length,"nucleobases.\n")

# Generators
name,seq = max(fasta_sequences("ecoli-proteome.faa"),key= lambda x: len(x[1]))
max_length = len(seq)
print("Generator:\nThe longest gene is",name,"and contains",max_length,"nucleobases.")

Lists:
The longest gene is gi|145698281|ref|NP_416485.4| putative adhesin [Escherichia coli str. K-12 substr. MG1655] and contains 2358 nucleobases.

Generator:
The longest gene is gi|145698281|ref|NP_416485.4| putative adhesin [Escherichia coli str. K-12 substr. MG1655] and contains 2358 nucleobases.


##### Part e  

**Now write a function write_fasta(outfile,header,sequence) that writes the sequence
with header to the opened file outfile in FASTA format.**

In [29]:
def write_fasta(outfile, header, sequence):
    outfile.write(">"+header+'\n')
    i = 0
    while i < len(sequence):
        outfile.write(sequence[i:i+69]+'\n')  # Sequence lines cannot be > 70 chars
        i += 69

##### Part f  

**Put the definitions of the previous functions in a single file named fastatools.py .**

### Exercise 2

**Write a script cdna.py that takes two command line parameters. The first is an input file
containing a DNA sequence in FASTA format. Read the sequence and generate the cDNA
(complementary DNA strand) and write it to the file given as second parameter.**

In [1]:
run cdna.py ecoli-genome.fna ecoli-genome-comp.fna

### Exercise 3

##### Part a  

**Write a script orf_finder.py that takes two command line parameters. The first is an
input file containing a DNA sequence. The second is an output file that should contain
all the (longest) open reading frames found in DNA (from both strands).**  
  
**Write the open reading frames in the order the appear on the genome of ecoli-genome.fna
and store them in a file ecoli-orf.ffn.**

In [2]:
run orf_finder.py ecoli-genome.fna ecoli-orf.ffn

#### Part b

Here I made my orf_finder function in the script into a generator and iterated over each ORF while writing thme to the specified file

### Exercise 4

##### Part a  

**Write a function get_sequence_positions(fasta_file) that can retrieve the sequence positions from the headers of the sequences in ecoli-orf.ffn and ecoli-genes-standard.ffn. The results should be stored in a dictionary where the key is the end position and the value is the start positon.**

In [2]:
def get_sequence_positions(fasta_file):
    with open(fasta_file, 'r') as f:
        line_list = f.read().split(':')[1:]  # Extract lines after ':' where numbers directly are

        # Get numbers from line_list and split by ',', ' ', '\n' using regex
        coding_sequences = [re.split(r'[, \n]', seq)[0] for seq in line_list]
        split_seq_numbers = []
        for seq_set in coding_sequences:
            numbers = seq_set.split('-')  # Remove '-' between numbers and put into tuple
            split_seq_numbers.append((numbers[0], numbers[1]))
        seq_dict = defaultdict(list)  # Create dictionary list to append seq positions to
        for set in split_seq_numbers:
            seq_dict[set[1]].append(set[0])  # Append start position to stop position key in dictionary
    return seq_dict

##### Part b  

Write a script correct_orfs.py taking two command line parameters, the first should be a FASTA file of open reading frames and the second a file of genes. It should calculate and print the following things:
* total number of open reading frames
* total number of genes
* total number and ratio of open reading frames correctly predicting a gene (with
correct start codon)
* total number and ratio of open reading frames correctly predicting at least the stop
codon of a gene (i.e. the ORF might be longer because the real start of transcription
is determined by another start codon in the ORF.)
* number of missed genes, i.e., genes of ecoli-genes-standard.ffn whose stop
codon does not correspond to any ORF stop codon.

In [3]:
run correct_orfs.py ecoli-orfs.ffn ecoli-genes.ffn

----------------------------------------------
Number of genes: 4321
Number of open reading frames: 81687
Number of ORFs predicting a gene: 3140
Ratio of ORFs predicting a gene: 0.038
Number of ORFs with matching gene stop codon: 4201
Ratio of ORFs with matching gene stop codon: 0.051
Number of genes with no matching ORF stop codon: 120


##### Part c  

Modify your previous script so that you can give an additional parameter determining the minimum length of an ORF/gene as a parameter. All ORFs smaller than the given parameter should not be considered genes. Check your results for minimum sizes of 50, 100, 150, 200, 250, 300, and 350 amino acids.

In [5]:
run correct_orfs.py ecoli-orfs.ffn ecoli-genes.ffn 50

----------------------------------------------
Number of genes: 4321
Number of open reading frames: 81687
Number of ORFs predicting a gene: 3084
Ratio of ORFs predicting a gene: 0.213
Number of ORFs with matching gene stop codon: 4098
Ratio of ORFs with matching gene stop codon: 0.283
Number of genes with no matching ORF stop codon: 223


In [6]:
run correct_orfs.py ecoli-orfs.ffn ecoli-genes.ffn 100

----------------------------------------------
Number of genes: 4321
Number of open reading frames: 81687
Number of ORFs predicting a gene: 2831
Ratio of ORFs predicting a gene: 0.473
Number of ORFs with matching gene stop codon: 3752
Ratio of ORFs with matching gene stop codon: 0.627
Number of genes with no matching ORF stop codon: 569


In [7]:
run correct_orfs.py ecoli-orfs.ffn ecoli-genes.ffn 150

----------------------------------------------
Number of genes: 4321
Number of open reading frames: 81687
Number of ORFs predicting a gene: 2498
Ratio of ORFs predicting a gene: 0.618
Number of ORFs with matching gene stop codon: 3301
Ratio of ORFs with matching gene stop codon: 0.816
Number of genes with no matching ORF stop codon: 1020


In [8]:
run correct_orfs.py ecoli-orfs.ffn ecoli-genes.ffn 200

----------------------------------------------
Number of genes: 4321
Number of open reading frames: 81687
Number of ORFs predicting a gene: 2153
Ratio of ORFs predicting a gene: 0.678
Number of ORFs with matching gene stop codon: 2845
Ratio of ORFs with matching gene stop codon: 0.896
Number of genes with no matching ORF stop codon: 1476


In [9]:
run correct_orfs.py ecoli-orfs.ffn ecoli-genes.ffn 250

----------------------------------------------
Number of genes: 4321
Number of open reading frames: 81687
Number of ORFs predicting a gene: 1797
Ratio of ORFs predicting a gene: 0.702
Number of ORFs with matching gene stop codon: 2371
Ratio of ORFs with matching gene stop codon: 0.926
Number of genes with no matching ORF stop codon: 1950


In [10]:
run correct_orfs.py ecoli-orfs.ffn ecoli-genes.ffn 300

----------------------------------------------
Number of genes: 4321
Number of open reading frames: 81687
Number of ORFs predicting a gene: 1437
Ratio of ORFs predicting a gene: 0.708
Number of ORFs with matching gene stop codon: 1899
Ratio of ORFs with matching gene stop codon: 0.935
Number of genes with no matching ORF stop codon: 2422


In [11]:
run correct_orfs.py ecoli-orfs.ffn ecoli-genes.ffn 350

----------------------------------------------
Number of genes: 4321
Number of open reading frames: 81687
Number of ORFs predicting a gene: 1092
Ratio of ORFs predicting a gene: 0.716
Number of ORFs with matching gene stop codon: 1438
Ratio of ORFs with matching gene stop codon: 0.942
Number of genes with no matching ORF stop codon: 2883
